##### Подготовка данных target_class

Порядок:
 
1. Чтение данных target_class    
2. Убрать крупных клиентов по сумме финансирования   
3. Выбор только нужных строк, попадающих в одну из выборок   
4. decision_date - дата принятия решения без времени
5. Отбор только легковой и грузовой транспорт
6. Метка агрегируется по максимальному в рамках одной сделки значению

In [1]:
pre = '../../' # путь до главной директории проекта

%run ../init.ipynb #подгрузить нужные библиотеки

In [2]:
#  исключить крупных клиентов ( по сумма сделки финансирования) из выборки

def filterMajorClients(df_target):
    
    df = df_target.copy()
    
    m = (df['DATE_ACTUAL']< '25.01.21') & ((df['NL_NEW_PRODUCTNAME']=='Express') | (df['O_NEW_TOTALFINANCINGSUM']<=10e+6))
    df.loc[m, 'маршрутизация_по_моделям'] = True              
    m = (df['DATE_ACTUAL'].between('25.01.21', '15.12.21')) & ((df['NL_NEW_PRODUCTNAME']=='Express') | (df['O_NEW_TOTALFINANCINGSUM']<=15e+6))
    df.loc[m, 'маршрутизация_по_моделям'] = True
    m = (df['DATE_ACTUAL'].between('15.12.21', '15.07.2022')) & ((df['NL_NEW_PRODUCTNAME']=='Express') | (df['O_NEW_TOTALFINANCINGSUM']<=20313824))
    df.loc[m, 'маршрутизация_по_моделям'] = True
    m = (df['DATE_ACTUAL'].between('15.07.22', '31.03.2023')) & ((df['NL_NEW_PRODUCTNAME']=='Express' )| (df['O_NEW_TOTALFINANCINGSUM']<=26711533))
    df.loc[m, 'маршрутизация_по_моделям'] = True
    m = (df['DATE_ACTUAL']>'31.03.2023') & ((df['NL_NEW_PRODUCTNAME']=='Express') | (df['O_NEW_TOTALFINANCINGSUM']<=5e+7))
    df.loc[m, 'маршрутизация_по_моделям'] = True
    
    df.loc[(df['маршрутизация_по_моделям']!=True), 'Попадает в выборку'] = None
    
    return df

In [3]:
def target_preprocessing(target_data):
    target_data.rename(columns={'credit_risk':'экспертный_кред_дефолт'}, inplace=True)
    target_data['экспертный_кред_дефолт'] = target_data['экспертный_кред_дефолт'].astype('int')

    target_data['Метка кред дефолта'] = target_data['Метка кред дефолта'].combine_first(target_data['экспертный_кред_дефолт'])
    # target_data['Метка фрод дефолта'] = target_data['combined_fraud_default']

    target_data = filterMajorClients(target_data)
    
    m1 = (target_data['Попадает в выборку']=='обе модели')
    target_data = target_data.loc[m1]

    # без ИНН контрагента не нужно
    mask = (target_data['ИНН контрагента'].notnull())
    target_data = target_data[mask]
    
    # без даты принятия решения по сделке не нужно
    mask = (target_data['Дата принятия решения по сделке'].notnull())
    target_data = target_data[mask]

    #  проверка что в 1 сделке только одни ИНН участвует
    target_data['ИНН_count'] = target_data.groupby('Номер сделки')['ИНН контрагента'].transform('nunique')
    assert len(target_data[target_data['ИНН_count']>1]) == 0
    
    return target_data   

In [4]:
# сюда таблицы служебные будем кидать, для отчетов
name = pre + "data/interm/auxilary/processing_target.xlsx"
writer = ExcelWriter(name)

In [5]:
name = pre + "data/interm/external_pkl/target_dataset.pkl"
target_class = pd.read_pickle(name, compression={'method':'zip'})

print(len(target_class))
# target_class.head(1)

252134


In [6]:
name = pre + "data/interm/external_pkl/mdl_app.pkl"
mdl_app = pd.read_pickle(name, compression={'method':'zip'})

mdl_app.drop_duplicates(subset='PL', inplace=True)

print(len(mdl_app))
# mdl_app.head(1)

250658


In [7]:
# пример строк
target_class.sample(100).to_excel(writer, sheet_name='target_class samples')

In [8]:
target_class.reset_index(inplace=True)
target_class.rename(columns = {'index':'PL'}, inplace=True)

In [9]:
target_data = pd.merge(target_class, mdl_app, on='PL', how='left')

target_data = target_preprocessing(target_data)
target_data['decision_date'] = target_data['Дата принятия решения по сделке'].dt.normalize()
# target_data.head(1)

берем только легковой и грузовой транспорт

In [10]:
print("общий список продуктов {}".format(target_data['Продукт'].unique()))

valid_products = ['Express', 'Стандарт', 'Грузовая техника', 'Такси', 'Такси+']
target_data = target_data[target_data['Продукт'].isin(valid_products)]

общий список продуктов ['Express' 'Стандарт' 'Автобусы' 'Самоходные машины' 'Грузовая техника'
 'Такси' 'Складская техника' 'Такси+']


метка будет агрегироваться по максимуму в рамках одной сделки при чем после того, как взяли только valid_products

In [11]:
# берем по обращению метку максимальную
target_data['Метка кред дефолта'] =  target_data.groupby(['Номер сделки', 'ИНН контрагента'])['Метка кред дефолта'].transform(max)

# и удалим дубли по номеру сделки. Это наши рассматриваемые таблица без дублей по номерам сделок
target_data = target_data.drop_duplicates(subset=['Номер сделки', 'ИНН контрагента']).reset_index()

In [12]:
keep_columns = ['PL', 'Номер сделки', 'Номер договора (кратко)', 'ИНН контрагента', 'Продукт',
                'Дата принятия решения по сделке', 'decision_date', 
                'Метка кред дефолта']

target_data[keep_columns].to_pickle(pre+"data/interm/target_data.pkl", compression={'method':'zip'})
print(len(target_data))
# target_data[keep_columns].head(1)

127968


In [13]:
# сохраняем tmp таблицы
if len(writer.sheets):
    writer.save() 